In [1]:
import os

In [2]:
%pwd

'/Users/iqbal/Desktop/LangWiz-AI/research'

In [7]:
os.chdir('../')

In [8]:
%pwd

'/Users/iqbal/Desktop/LangWiz-AI'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    result_dataset_path: Path
    

In [16]:
from LangWizAI.constants import *
from LangWizAI.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        print(config)
        
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
            result_dataset_path = config.result_dataset_path
        )
        return data_transformation_config

In [18]:
import os
from LangWizAI import logger
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk

In [43]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
        self.model = AutoModelForSeq2SeqLM.from_pretrained(config.tokenizer_name)
    
    def translate(self, example):
        # print(example)
        
        inputs = example['source_text']
        # print(inputs)
        targets = example['target_text']
        
        if not isinstance(inputs,(str,list)):
            inputs = str(inputs)
        if not isinstance(targets,(str,list)):
            targets = str(targets)
            
        # print(inputs)
        input_encodings = self.tokenizer(inputs, max_length = 128,
                                         truncation=True, padding=True,
                                         return_tensors = 'pt')
        with self.tokenizer.as_target_tokenizer():
            target_encodings= self.tokenizer(targets, max_length = 128,
                                         truncation=True, padding=True,
                                         return_tensors = 'pt')
        
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask' : input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']       
        }
    
    def convert(self):
        data = load_dataset('csv', data_dir=Path(self.config.data_path))
        data_pt = data.map(self.translate, batched =True)
        data_pt.save_to_disk(self.config.result_dataset_path)

In [44]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config = data_transformation_config)
data_transformation.convert()

[2024-07-27 07:07:54,271: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-27 07:07:54,272: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-27 07:07:54,273: INFO: common: created directory at: artifacts]
[2024-07-27 07:07:54,273: INFO: common: created directory at: artifacts/data_transformation]
{'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data/europarl/processed/fr_en', 'result_dataset_path': 'artifacts/data_transformation/europarl/processed/fr_e', 'tokenizer_name': 'Helsinki-NLP/opus-mt-fr-en'}


Map:   0%|          | 0/1905251 [00:00<?, ? examples/s]

Saving the dataset (0/8 shards):   0%|          | 0/1905251 [00:00<?, ? examples/s]